# Линейная регрессия, полиноминальная, Lasso, Ridge и ElasticNet 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
from pprint import pprint, pformat
import copy
from pathlib import Path

import joblib

from my_lib import *
from my_config import *

In [2]:
from sklearn.linear_model import LinearRegression # для построения моделей линейной регрессии
from sklearn.preprocessing import PolynomialFeatures # для преобразования исходных признаков в полиномиальные, для построения моделей полиномиальной регрессии

from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer, PowerTransformer

In [4]:
#dataset_df = joblib.load(Path(result_foler, dataset_filename_after_PrepareTarget))
params = joblib.load(Path(result_foler, params_filename_after_PrepareTarget))

In [5]:
#dataset_df_X  = dataset_df.drop([params["target_column"]], axis=1)
#dataset_df_Y  = dataset_df[params["target_column"]]

In [6]:
scaler_name = 'NoScaler'
X_train =joblib.load(Path(result_foler, X_train_template_filename_after_split % scaler_name))
X_test = joblib.load(Path(result_foler, X_test_template_filename_after_split % scaler_name))
y_train =joblib.load(Path(result_foler, y_train_template_filename_after_split % scaler_name))
y_test = joblib.load(Path(result_foler, y_test_template_filename_after_split % scaler_name))


In [ ]:
# Масштабирование
scaler = StandardScaler()
X_scaled = scaler.fit_transform(dataset_df_X)

# Добавление полиномиальных признаков
poly = PolynomialFeatures(degree=3)
X_poly = poly.fit_transform(X_scaled)



# Обучение модели
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Оценка
print(f'R² score: {r2_score(y_test, y_pred):.3f}')

In [ ]:
models_scores = pd.DataFrame(columns=["r2_score_train", "r2_score_test",
                                      "mse_train", "mse_test", 
                                      "rmse_train", "rmse_test", 
                                      "mae_train", "mae_test",
                                      "coef", "params"
                                      ])

In [ ]:
# задаем интервал перебора для alpha
# альфа зависит от размера выборки

start = 10      # Начальное значение диапазона
stop = 1000       # Конечное значение диапазона (не включается в результат)
step = 10       # Шаг между значениями
float_range = np.arange(start, stop, step)
print(float_range)

In [ ]:
# добавить масшабирование признаков и целевых переменных
scalers = {'NoScaler': None,
           'StandardScaler': StandardScaler(),
           'MinMaxScaler': MinMaxScaler(),
           'MaxAbsScaler': MaxAbsScaler(),
           'RobustScaler': RobustScaler()
          }

In [ ]:
%%time
scalers = ['NoScaler', 'StandardScaler', 'MinMaxScaler', 'MaxAbsScaler', 'RobustScaler']
#scalers = ['StandardScaler']
#scalers = ['MinMaxScaler']
#scalers = ['MaxAbsScaler']
#scalers = ['RobustScaler']
#scalers = ['NoScaler']
for scaler_name in scalers:
    X_train =joblib.load(Path(result_foler, X_train_template_filename_after_split % scaler_name))
    X_test = joblib.load(Path(result_foler, X_test_template_filename_after_split % scaler_name))
    y_train =joblib.load(Path(result_foler, y_train_template_filename_after_split % scaler_name))
    y_test = joblib.load(Path(result_foler, y_test_template_filename_after_split % scaler_name))

    # -------------------- LinearRegression() --------------------
    lin_reg = LinearRegression()
    lin_reg_fit = lin_reg.fit(X_train, y_train)
    Y_pred_train_lin = lin_reg_fit.predict(X_train) # train
    Y_pred_test_lin = lin_reg_fit.predict(X_test) # test
    add_scores(models_scores, f"{scaler_name}_LinearRegression", 
               y_train, Y_pred_train_lin, y_test, Y_pred_test_lin, 
               pformat(lin_reg_fit.get_params()), f"{lin_reg_fit.coef_}, const={lin_reg_fit.intercept_}")
    #display(models_scores)

    # -------------------- PolynomialFeatures() --------------------
    for d in [2, 3]:
        poly_features = PolynomialFeatures(degree=d) # степень до 7, долго
        X_train_poly = poly_features.fit_transform(X_train)
        X_test_poly = poly_features.transform(X_test)
        poly_reg = LinearRegression()
        poly_reg_fit = poly_reg.fit(X_train_poly, y_train)    
        
        Y_pred_train_poly = poly_reg_fit.predict(X_train_poly)
        Y_pred_test_poly = poly_reg_fit.predict(X_test_poly)    

        add_scores(models_scores, f"{scaler_name}_PolynomialFeatures(degree={d})", 
                   y_train, Y_pred_train_poly, y_test, Y_pred_test_poly, 
                   pformat(poly_reg_fit.get_params()), f"{poly_reg_fit.coef_}, const={poly_reg_fit.intercept_}")
        #display(models_scores)
        

    # -------------------- ElasticNet() --------------------
    param_grid = {
        'alpha': [0.00005, 0.0005, 0.001, 0.01, 0.05, 0.06, 0.08, 1, 2, 3],
        'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    }
    elastic_net = GridSearchCV(ElasticNet(), param_grid, scoring='r2', cv=10)        
    res_elastic_net_model = elastic_net.fit(X_train, y_train)

    # построим регрессию гребневую L2 с оптимальным параметром регуляризации, который мы подобрали перебором
    model_reg_elastic = ElasticNet(max_iter=1000, **res_elastic_net_model.best_params_) # alpha — величина регуляризации

    # обучение
    model_reg_elastic.fit(X_train, y_train)
    y_pred_test_elastic = model_reg_elastic.predict(X_test)
    y_pred_train_elastic = model_reg_elastic.predict(X_train)

    add_scores(models_scores, f"{scaler_name}_ElasticNet", 
               y_train, y_pred_train_elastic, y_test, y_pred_test_elastic, 
               pformat(res_elastic_net_model.best_params_), f"{model_reg_elastic.coef_}, const={model_reg_elastic.intercept_}")
    #display(models_scores)
    
    # -------------------- Ridge() --------------------
    param_grid = {
        'alpha': float_range
    }

    ridge = GridSearchCV(Ridge(), param_grid, scoring='r2', cv=10)        
    res_ridge_model = ridge.fit(X_train, y_train)

    # построим регрессию гребневую L2 с оптимальным параметром регуляризации, который мы подобрали перебором
    model_ridge = Ridge(max_iter=1000, **res_ridge_model.best_params_) # alpha — величина регуляризации

    # обучение
    model_ridge.fit(X_train, y_train)
    y_pred_test_elastic = model_ridge.predict(X_test)
    y_pred_train_elastic = model_ridge.predict(X_train)

    add_scores(models_scores, f"{scaler_name}_Ridge", 
               y_train, y_pred_train_elastic, y_test, y_pred_test_elastic, 
               pformat(res_ridge_model.best_params_), f"{model_ridge.coef_}, const={model_ridge.intercept_}")
    #display(models_scores)    
    
    # -------------------- Lasso() --------------------
    param_grid = {
        'alpha': float_range
    }

    lasso = GridSearchCV(Lasso(), param_grid, scoring='r2', cv=10)        
    res_lasso_model = lasso.fit(X_train, y_train)

    # построим регрессию гребневую L2 с оптимальным параметром регуляризации, который мы подобрали перебором
    model_lasso = Lasso(max_iter=1000, **res_lasso_model.best_params_) # alpha — величина регуляризации

    # обучение
    model_lasso.fit(X_train, y_train)
    y_pred_test_elastic = model_lasso.predict(X_test)
    y_pred_train_elastic = model_lasso.predict(X_train)

    add_scores(models_scores, f"{scaler_name}_Lasso", 
               y_train, y_pred_train_elastic, y_test, y_pred_test_elastic, 
               pformat(res_lasso_model.best_params_), f"{model_lasso.coef_}, const={model_lasso.intercept_}")

    display(models_scores)

In [ ]:
display(models_scores)

In [ ]:
print("Коэффициенты моделей:")
for i in models_scores.index:
    print(f'{i}: {models_scores.loc[i]["coef"]}')

In [ ]:
print("Гиперпараметры моделей:")
for i in models_scores.index:
    print(f'{i}: {models_scores.loc[i]["params"]}')

In [ ]:
print("r2_score моделей:")
for i in models_scores.index:
    print(f'{i:45}| train: {round(models_scores.loc[i]["r2_score_train"],8):10}  | test: {round(models_scores.loc[i]["r2_score_test"],8):10}')

В результате были построены модели:
  * линейная регрессия
  * полиноминальная регрессия с degree=2, 3
  * ElasticNet с подбором гиперпараметров
  * Ridge с подбором гиперпараметров
  * Lasso с подбором гиперпараметров

для нескольких наборов данных, отличающихся примененным алгоритмом нормализации данных - без нормализации, 'StandardScaler', 'MinMaxScaler', 'MaxAbsScaler', 'RobustScaler'.

Также проверялись разные подходы к подготовке набора данных:
  * с удалением столбцов с большой долей пропусков
  * с заполнением пропусков модой и с удалением строк, в которых есть пропуски
  * разные настройки OneHotEncoder


Результат был примерно один и тот же. В частности метрика r2_score колебалась в диапазоне от 0.0016 до 0.0040, что крайне мало и показывает, что построенные модели непригодны для прогнозирования целевой функции.
Следовательно:
* либо методы регрессии не подходят к решению этой задаче
* либо где-то ошибка в коде
* либо в принципе неверный подход к подготовке данных и/или обучению моделей
* либо неверно интерпретирую метрики

Предполагаю, что какая-то из двух последних причин. Нужна обратная связь.
